In [0]:
import os
os.environ["CUDA_ROOT"]='/usr/local/cuda'
os.environ["PYTENSOR_FLAGS"]='allow_gc=False' #,floatX=float64'
#from pytensor.configdefaults import config
import pymc as pm
import pymc_bart as pmb
import matplotlib.pyplot as plt
import numpy as np

import numpy as np
import sklearn as skl
import scipy.stats as sp
#import simsurv_func as ssf
import mlflow
# import pyspark

In [0]:
# M = 200 # number of trees
# DRAWS = 200
# TUNE = 100
# CORES = 4
# SPLIT_RULES = "[pmb.ContinuousSplitRule(), pmb.ContinuousSplitRule(), pmb.OneHotSplitRule(), pmb.OneHotSplitRule()]"

experiment_id=dbutils.jobs.taskValues.get("cdh-ml-model",
                                          "experiment_id",
                                          debugValue=2221480985583347)

#run_name = dbutils.jobs.taskValues.get("cdh-ml-init", 
#                                         "run_name", 
#                                         debugValue="test2")

#run_id = dbutils.jobs.taskValues.get("cdh-ml-run",
#                                    "run_id_main",
#                                    debugValue = "5c4b0bab2668466ea9ac022e482adc35")

ptask="cdh-ml-bart-init"
M = dbutils.jobs.taskValues.get(ptask, "M", debugValue=200)
DRAWS = dbutils.jobs.taskValues.get(ptask, "DRAWS", debugValue=200)
TUNE = dbutils.jobs.taskValues.get(ptask, "TUNE", debugValue=200)
CORES = dbutils.jobs.taskValues.get(ptask, "CORES", debugValue=4)
SPLIT_RULES = dbutils.jobs.taskValues.get(ptask, "SPLIT_RULES", debugValue="[pmb.ContinuousSplitRule(), pmb.ContinuousSplitRule(), pmb.OneHotSplitRule(), pmb.OneHotSplitRule()]")

SEED=2
ALPHA=0.95

DATA_URL="dbfs:/databricks/mlflow-tracking/1348027641275128/08c1ac0ba2124291922209acebec94cb/artifacts/lung.json"
DATA_RUN_ID='08c1ac0ba2124291922209acebec94cb'

In [0]:
#mlflow.set_experiment(experiment_id=experiment_id)

In [0]:
lung = mlflow.load_table(
    artifact_file="data/lung_tte_bart.json",
    run_ids=[DATA_RUN_ID],
    # Append a column containing the associated run ID for each row
   # extra_columns=["run_id"],
)
print(lung)

# connected to the tracking_server_url
mlflow_client = mlflow.tracking.MlflowClient()

# list all experiment at this Tracking server
# mlflow_client.list_experiments()

# extract params/metrics data for run `test_run_id` in a single dict 
run_data_dict = mlflow_client.get_run(DATA_RUN_ID).data.to_dictionary()

#
 #   lung = pd.read_json(io.BytesIO(f.read()))
#lung = pd.read_json(DATA_URL)
#lung.head()

In [0]:
print(run_data_dict.keys())

In [0]:
# configure analytic dataset
lung["karno"] = lung["ph.karno"].fillna(lung["pat.karno"])
## adjust time to months
#lung["time2"] = np.ceil(lung["time"]/30)
#lung["sex2"] = lung["sex"]-1
#time = lung.time2
event = lung.status == 2

# karno try categorical and continuous
x = pd.concat([time, lung[["age","sex2","karno"]]], axis=1)


In [0]:

# sklearn set-up
y_sk = ssf.get_y_sklearn(delta, time)
x_sk = lung[["age","sex2","karno"]]

# long time
b_tr_t, b_tr_delta, b_tr_x = ssf.surv_pre_train2(data_x_n = x_sk, data_y=y_sk, X_TIME=True)

# create the counterfactual test dataset
# First half is male second half is
b_te_x = ssf.get_bart_test(x_out = x_sk, T = np.unique(b_tr_t))
b_te_x[:,2] = 0
b_te_x2 = b_te_x.copy()
b_te_x2[:,2] = 1
b_te_x3 = np.concatenate([b_te_x, b_te_x2], axis=0)

In [0]:
# BART
# M = 200 # number of trees
# DRAWS = 2000
# TUNE = 1000
# CORES = 4
# SPLIT_RULES = "[pmb.ContinuousSplitRule(), pmb.ContinuousSplitRule(), pmb.OneHotSplitRule(), pmb.OneHotSplitRule()]"

    # run pymc
offset = sp.norm.ppf(np.mean(b_tr_delta))
with pm.Model() as bart:
    x_data = pm.MutableData("x", b_tr_x)
    f = pmb.BART("f", X=x_data, Y=b_tr_delta, m=M, alpha = ALPHA, split_rules=eval(SPLIT_RULES))
    z = pm.Deterministic("z", f + offset)
    mu = pm.Deterministic("mu", pm.math.invprobit(z))
    y_pred = pm.Bernoulli("y_pred", p=mu, observed=b_tr_delta, shape=x_data.shape[0])
    bdata = pm.sample(random_seed=SEED, draws=DRAWS, tune = TUNE, cores=CORES)

#akb add
pm.model_to_graphviz(bart)

In [0]:
with bart:
# pm.set_data({"x":pd.DataFrame(test_x), "off":off_test})
    pm.set_data({"x":pd.DataFrame(b_te_x3)})
    pp = pm.sample_posterior_predictive(bdata, var_names = ["y_pred", "f", "z", "mu"])


In [0]:
with mlflow.start_run(experiment_id=experiment_id, run_id=run_id) as run:

    # get survival
    x_out = np.concatenate([x_sk.to_numpy(), x_sk.to_numpy()], axis=0)
    bart_sv_fx = ssf.get_sv_fx(pp, x_out)

    # get the original and counterfactual
    og_shp = x_sk.shape[0]
    or_bart_sv_fx = bart_sv_fx[0:og_shp,:]
    cf_bart_sv_fx = bart_sv_fx[og_shp:, :]

    # get mean and quantile
    or1 = or_bart_sv_fx.mean(axis=0)
    orp = np.quantile(or_bart_sv_fx, q=[0.05,0.95], axis=0)
    cf1 = cf_bart_sv_fx.mean(axis=0)
    cfp = np.quantile(cf_bart_sv_fx, q=[0.05,0.95], axis=0)


    plt_time = np.unique(b_tr_t)

    # plot
    fig = plt.figure()
    plt.step(plt_time, or1, label = "male", color="darkblue")
    plt.step(plt_time, orp[0], color="darkblue", alpha=.4)
    plt.step(plt_time, orp[1], color="darkblue", alpha=.4)
    plt.step(plt_time, cf1, label = "female", color="darkorange")
    plt.step(plt_time, cfp[0], color="darkorange", alpha=.4)
    plt.step(plt_time, cfp[1], color="darkorange", alpha=.4)
    plt.legend()
    mlflow.log_figure(fig, "male_female.png")